In [1]:
from IPython.display import display, Markdown, Latex
import pandas as pd
import numpy as np
import json
import os

## Experiments and Functions

In [2]:
experiments = [
            {'id': '1', 'source':'imdb', 'target':'uwcse', 'predicate':'workedunder', 'to_predicate':'advisedby', 'arity': 2},
            #{'id': '2', 'source':'uwcse', 'target':'imdb', 'predicate':'advisedby', 'to_predicate':'workedunder', 'arity': 2},
            {'id': '3', 'source':'imdb', 'target':'cora', 'predicate':'workedunder', 'to_predicate':'samevenue', 'arity': 2},
            {'id': '4', 'source':'cora', 'target':'imdb', 'predicate':'samevenue', 'to_predicate':'workedunder', 'arity': 2},
            ##{'id': '5', 'source':'cora', 'target':'imdb', 'predicate':'sametitle', 'to_predicate':'workedunder', 'arity': 2},
            ##{'id': '6', 'source':'imdb', 'target':'cora', 'predicate':'workedunder', 'to_predicate':'sametitle', 'arity': 2},
            ##{'id': '5', 'source':'uwcse', 'target':'cora', 'predicate':'advisedby', 'to_predicate':'samevenue', 'arity': 2},
            ##{'id': '6', 'source':'cora', 'target':'uwcse', 'predicate':'samevenue', 'to_predicate':'advisedby', 'arity': 2},
            {'id': '7', 'source':'yeast', 'target':'twitter', 'predicate':'proteinclass', 'to_predicate':'accounttype', 'arity': 2},
            {'id': '8', 'source':'twitter', 'target':'yeast', 'predicate':'accounttype', 'to_predicate':'proteinclass', 'arity': 2},
            {'id': '9', 'source':'nell_sports', 'target':'nell_finances', 'predicate':'teamplayssport', 'to_predicate':'companyeconomicsector', 'arity': 2},
            {'id': '10', 'source':'nell_finances', 'target':'nell_sports', 'predicate':'companyeconomicsector', 'to_predicate':'teamplayssport', 'arity': 2},
]

In [3]:
def load_data(experiments, metric, directory):
    data = {}
    for item in experiments:
        experiment_title = item['id'] + '_' + item['source'] + '_' + item['target']
        
        if(metric != ''):
            file_path = os.getcwd() + directory + experiment_title + '/' + experiment_title + '_fasttext_{}.json'.format(metric)
        else:
            file_path = os.getcwd() + directory + experiment_title + '/' + experiment_title + '.json'
        try:
            with open(file_path, 'r') as fp:
                results = json.load(fp)

                data[experiment_title] = results
        except:
            continue
    return data

## Estudo dos Mapeamentos

### Depth-First Mapping

In [4]:
# Setting paths parameters
mapping_order = 'deep'
k = 1
compare_stopwords = False

#Removing Yeast and Twitter pair of experiments because there is no stopwords in predicates
#exp = experiments.copy()
#exp.remove({'id': '7', 'source':'yeast', 'target':'twitter', 'predicate':'proteinclass', 'to_predicate':'accounttype', 'arity': 2})
#exp.remove({'id': '8', 'source':'twitter', 'target':'yeast', 'predicate':'accounttype', 'to_predicate':'proteinclass', 'arity': 2})

In [11]:
# Using stopwords
path = f'/results/transfer/{mapping_order}/w_stopwords/k_{k}/'
data_softcosine_stopwords = load_data(experiments, 'softcosine', path)
data_euclidean_stopwords = load_data(experiments, 'euclidean', path)
data_wmd_stopwords = load_data(experiments, 'wmd', path)
data_rwmd_stopwords = load_data(experiments, 'relax-wmd', path)

if compare_stopwords:
    # Removing stopwords
    path = f'/results/transfer/{mapping_order}/no_stopwords/k_{k}/'
    data_softcosine_no_stopwords = load_data(experiments, 'softcosine', path)
    data_euclidean_no_stopwords = load_data(experiments, 'euclidean', path)
    data_wmd_no_stopwords = load_data(experiments, 'wmd', path)
    data_rwmd_no_stopwords = load_data(experiments, 'relax-wmd', path)

treeboostler_path = f'/results/baselines/transfer/'
#treeboostler_data = load_data(experiments, '', treeboostler_path)
#rdnb_data = load_data(experiments, '', treeboostler_path + '/rdn_b/')

for metric in ['CLL', 'AUC ROC', 'AUC PR', 'Learning time', 'Mapping time']: #, 'Recall', 'F1', 'Precision', 'Learning and Revision time', 'Inference time']:
    display(Markdown('# Results for ' + metric))
    table = []
    for j in range(len(experiments)):
            
        dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
        if dataset in data_wmd_stopwords:
            mapping_time = ''
            results_using_stopwords = {}
            results_removing_stopwords = {}

            # Results using Stopwords
            results_using_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
            results_using_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
            results_using_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
            results_using_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])

            #results_using_stopwords['softcosine no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
            #results_using_stopwords['euclidean no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
            #results_using_stopwords['wmd no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
            #results_using_stopwords['relax-wmd no revision'] = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])

            if compare_stopwords:
                # Results removing stopwords
                results_removing_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_no_stopwords[str(dataset)]])
                results_removing_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_no_stopwords[str(dataset)]])
                results_removing_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_no_stopwords[str(dataset)]])
                results_removing_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_no_stopwords[str(dataset)]])

            #if(experiments[j]['id'] != '10'):
                #treeboostler = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in treeboostler_data[str(dataset)]])
                #treeboostler_no_revision = np.array([(np.array([item['transfer']['parameter'][metric] for item in sublist])).mean() for sublist in treeboostler_data[str(dataset)]])
                #print(treeboostler)
            #rdnb = np.array([(np.array([item['rdn-b'][metric] for item in sublist])).mean() for sublist in rdnb_data[str(dataset)]])
            
            if(experiments[j]['id'] != '10'):
                table.append([dataset,
                #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                #'%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
                '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
                             ])
            else:
                table.append([dataset,
                #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                #'NaN +/- NaN',
                '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
              ])
            
            #table.append(['', '', '', '', '', '', ''])
           
            if(experiments[j]['id'] != '10' and compare_stopwords):
                table.append([dataset + '*',
                #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                #'%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),              
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),                
                             ])
            elif(compare_stopwords):
                table.append([dataset + '*',
                #'%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                #'NaN +/- NaN',
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),
                             ])
            
            # No Revision
            #table.append([dataset + '_NR',
            #    '%.3f +/- %.3f' % (results_using_stopwords['softcosine no revision'].mean(), 2 * results_using_stopwords['softcosine'].std()),
            #    '%.3f +/- %.3f' % (results_using_stopwords['euclidean no revision'].mean(), 2 * results_using_stopwords['euclidean'].std()),
            #    '%.3f +/- %.3f' % (results_using_stopwords['wmd no revision'].mean(), 2 * results_using_stopwords['wmd'].std()),
            #    '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd no revision'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
            #    '%.3f +/- %.3f' % (treeboostler_no_revision.mean(), 2 * treeboostler_no_revision.std()),
            #    '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
            #])
            #table.append(['', '', '', '', '', '', ''])

    #display(pd.DataFrame(table, columns=['Experiment', 'RDN-B'])) #, 'Transfer Learning with Revision Theory - Relax WMD'])) #, 'Learning from scratch (RDN-B)']))
    display(pd.DataFrame(table, columns=['Experiment', 'Transfer Learning - Soft Cosine', 'Transfer Learning - Euclidean', 'Transfer Learning - WMD', 'Transfer Learning - RWMD'])) #, 'Learning from scratch (RDN-B)']))
    #display(pd.DataFrame(table, columns=['Experiment', 'Transfer Learning', 'Transfer Learning with Revision Theory']))

# Results for CLL

,Experiment,Transfer Learning - Soft Cosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - RWMD
0,1_imdb_uwcse,-0.255 +/- 0.023,-0.254 +/- 0.007,-0.247 +/- 0.014,-0.250 +/- 0.014
1,3_imdb_cora,-0.675 +/- 0.023,-0.677 +/- 0.037,-0.668 +/- 0.033,-0.662 +/- 0.036
2,4_cora_imdb,-0.074 +/- 0.003,-0.076 +/- 0.003,-0.076 +/- 0.004,-0.074 +/- 0.003
3,7_yeast_twitter,-0.127 +/- 0.056,-0.107 +/- 0.008,-0.107 +/- 0.006,-0.101 +/- 0.002
4,8_twitter_yeast,-0.280 +/- 0.102,-0.282 +/- 0.115,-0.240 +/- 0.012,-0.224 +/- 0.031
5,9_nell_sports_nell_finances,-0.321 +/- 0.011,-0.320 +/- 0.011,-0.315 +/- 0.024,-0.325 +/- 0.020
6,10_nell_finances_nell_sports,-0.117 +/- 0.046,-0.137 +/- 0.071,-0.136 +/- 0.143,-0.085 +/- 0.001


# Results for AUC ROC

,Experiment,Transfer Learning - Soft Cosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - RWMD
0,1_imdb_uwcse,0.936 +/- 0.007,0.936 +/- 0.006,0.936 +/- 0.007,0.938 +/- 0.005
1,3_imdb_cora,0.599 +/- 0.046,0.589 +/- 0.059,0.600 +/- 0.047,0.603 +/- 0.050
2,4_cora_imdb,1.000 +/- 0.000,0.999 +/- 0.001,0.999 +/- 0.002,1.000 +/- 0.001
3,7_yeast_twitter,0.994 +/- 0.001,0.994 +/- 0.000,0.994 +/- 0.000,0.994 +/- 0.000
4,8_twitter_yeast,0.920 +/- 0.096,0.894 +/- 0.150,0.953 +/- 0.027,0.965 +/- 0.030
5,9_nell_sports_nell_finances,0.721 +/- 0.044,0.750 +/- 0.047,0.741 +/- 0.041,0.713 +/- 0.085
6,10_nell_finances_nell_sports,0.993 +/- 0.000,0.947 +/- 0.064,0.948 +/- 0.128,0.993 +/- 0.001


# Results for AUC PR

,Experiment,Transfer Learning - Soft Cosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - RWMD
0,1_imdb_uwcse,0.284 +/- 0.045,0.275 +/- 0.035,0.274 +/- 0.045,0.281 +/- 0.043
1,3_imdb_cora,0.464 +/- 0.044,0.453 +/- 0.054,0.463 +/- 0.042,0.464 +/- 0.045
2,4_cora_imdb,1.000 +/- 0.000,0.927 +/- 0.051,0.948 +/- 0.131,0.989 +/- 0.042
3,7_yeast_twitter,0.382 +/- 0.096,0.389 +/- 0.028,0.374 +/- 0.034,0.400 +/- 0.041
4,8_twitter_yeast,0.169 +/- 0.266,0.325 +/- 0.072,0.282 +/- 0.033,0.344 +/- 0.077
5,9_nell_sports_nell_finances,0.087 +/- 0.025,0.069 +/- 0.005,0.079 +/- 0.005,0.071 +/- 0.005
6,10_nell_finances_nell_sports,0.316 +/- 0.027,0.248 +/- 0.030,0.243 +/- 0.078,0.276 +/- 0.017


# Results for Learning time

,Experiment,Transfer Learning - Soft Cosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - RWMD
0,1_imdb_uwcse,5.944 +/- 0.446,6.440 +/- 1.441,5.896 +/- 0.551,27.233 +/- 0.326
1,3_imdb_cora,51.184 +/- 4.679,52.610 +/- 9.274,54.442 +/- 9.476,76.083 +/- 9.919
2,4_cora_imdb,4.366 +/- 0.119,4.428 +/- 0.057,4.439 +/- 0.079,30.180 +/- 0.059
3,7_yeast_twitter,23.388 +/- 41.322,26.098 +/- 49.435,24.552 +/- 43.776,84.739 +/- 3.020
4,8_twitter_yeast,20.395 +/- 31.988,13.056 +/- 24.364,19.640 +/- 8.262,34.115 +/- 9.097
5,9_nell_sports_nell_finances,62.705 +/- 10.765,54.046 +/- 11.151,53.814 +/- 8.930,93.787 +/- 26.359
6,10_nell_finances_nell_sports,331.293 +/- 186.915,350.767 +/- 223.203,298.125 +/- 69.603,298.071 +/- 60.389


# Results for Mapping time

,Experiment,Transfer Learning - Soft Cosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - RWMD
0,1_imdb_uwcse,0.025 +/- 0.005,0.394 +/- 1.521,0.017 +/- 0.000,21.375 +/- 0.008
1,3_imdb_cora,0.025 +/- 0.010,0.014 +/- 0.003,0.016 +/- 0.001,20.971 +/- 0.001
2,4_cora_imdb,0.021 +/- 0.007,0.012 +/- 0.000,0.015 +/- 0.001,26.234 +/- 0.002
3,7_yeast_twitter,0.031 +/- 0.010,0.794 +/- 1.565,0.018 +/- 0.001,27.652 +/- 0.002
4,8_twitter_yeast,0.032 +/- 0.018,0.010 +/- 0.001,0.017 +/- 0.000,10.719 +/- 0.001
5,9_nell_sports_nell_finances,0.182 +/- 0.038,0.059 +/- 0.036,0.137 +/- 0.001,39.277 +/- 0.003
6,10_nell_finances_nell_sports,0.151 +/- 0.001,0.562 +/- 1.440,0.143 +/- 0.003,42.719 +/- 0.003


### Ranked-First Mapping

In [24]:
# Setting paths parameters
mapping_order = 'most_similar'
k = '1'

compare_stopwords = False

In [26]:
# Using stopwords
path = f'/results/transfer/{mapping_order}/w_stopwords/k_{k}/'
data_softcosine_stopwords = load_data(experiments, 'softcosine', path)
data_euclidean_stopwords = load_data(experiments, 'euclidean', path)
data_wmd_stopwords = load_data(experiments, 'wmd', path)
data_rwmd_stopwords = load_data(experiments, 'relax-wmd', path)

if compare_stopwords:
    # Removing stopwords
    path = f'/results/transfer/{mapping_order}/no_stopwords/k_{k}/'
    data_softcosine_no_stopwords = load_data(experiments, 'softcosine', path)
    data_euclidean_no_stopwords = load_data(experiments, 'euclidean', path)
    data_wmd_no_stopwords = load_data(experiments, 'wmd', path)
    data_rwmd_no_stopwords = load_data(experiments, 'relax-wmd', path)

baseline_path = f'/results/baselines/transfer/'
baseline_data = load_data(experiments, '', baseline_path)

for metric in ['CLL', 'AUC ROC', 'AUC PR', 'Learning time']: #, 'Recall', 'F1', 'Precision', 'Learning and Revision time', 'Inference time']:
    display(Markdown('# Results for ' + metric))
    table = []
    
    for j in range(len(experiments)):
            
        dataset = experiments[j]['id'] + '_' + experiments[j]['source'] + '_' + experiments[j]['target']
        if dataset in data_euclidean_stopwords:
            mapping_time = ''
            results_using_stopwords = {}
            results_removing_stopwords = {}

            # Results using Stopwords
            results_using_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_stopwords[str(dataset)]])
            results_using_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_stopwords[str(dataset)]])
            results_using_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_stopwords[str(dataset)]])
            results_using_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_stopwords[str(dataset)]])
            
            if compare_stopwords:
                # Results removing stopwords
                results_removing_stopwords['softcosine'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_softcosine_no_stopwords[str(dataset)]])
                results_removing_stopwords['euclidean'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_euclidean_no_stopwords[str(dataset)]])
                results_removing_stopwords['wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_wmd_no_stopwords[str(dataset)]])
                results_removing_stopwords['relax-wmd'] = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in data_rwmd_no_stopwords[str(dataset)]])
            
            if(experiments[j]['id'] != '10'):
                treeboostler = np.array([(np.array([item['transfer'][metric] for item in sublist])).mean() for sublist in baseline_data[str(dataset)]])
                #treeboostler_no_revision = np.array([(np.array([item['transfer_{}'.format(amount)]['parameter'][metric] for item in sublist])).mean() for sublist in treeboostler_data[str(dataset)]])
                #print(treeboostler)
            rdnb = np.array([(np.array([item['rdn_b'][metric] for item in sublist])).mean() for sublist in baseline_data[str(dataset)]])
            
            if(experiments[j]['id'] != '10'):
                table.append([dataset,
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                '%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),
                '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
                             ])
            else:
                table.append([dataset,
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                'NaN +/- NaN',
                '%.3f +/- %.3f' % (results_using_stopwords['softcosine'].mean(), 2 * results_using_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['euclidean'].mean(), 2 * results_using_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['wmd'].mean(), 2 * results_using_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_using_stopwords['relax-wmd'].mean(), 2 * results_using_stopwords['relax-wmd'].std()),
              ])
            
            #table.append(['', '', '', '', '', '', ''])
           
            if(experiments[j]['id'] != '10' and compare_stopwords):
                table.append([dataset + '*',
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                '%.3f +/- %.3f' % (treeboostler.mean(), 2 * treeboostler.std()),              
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),                
                             ])
            elif(compare_stopwords):
                table.append([dataset + '*',
                '%.3f +/- %.3f' % (rdnb.mean(), 2 * rdnb.std()),
                'NaN +/- NaN',
                '%.3f +/- %.3f' % (results_removing_stopwords['softcosine'].mean(), 2 * results_removing_stopwords['softcosine'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['euclidean'].mean(), 2 * results_removing_stopwords['euclidean'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['wmd'].mean(), 2 * results_removing_stopwords['wmd'].std()),
                '%.3f +/- %.3f' % (results_removing_stopwords['relax-wmd'].mean(), 2 * results_removing_stopwords['relax-wmd'].std()),
                             ])

    #display(pd.DataFrame(table, columns=['Experiment', 'RDN-B'])) #, 'Transfer Learning with Revision Theory - Relax WMD'])) #, 'Learning from scratch (RDN-B)']))
    display(pd.DataFrame(table, columns=['Experiment','RDN-B', 'TreeBoostler', 'Transfer Learning - SoftCosine', 'Transfer Learning - Euclidean', 'Transfer Learning - WMD', 'Transfer Learning - Relax-WMD'])) #, 'Learning from scratch (RDN-B)']))
    #display(pd.DataFrame(table, columns=['Experiment', 'Transfer Learning', 'Transfer Learning with Revision Theory']))

# Results for CLL

,Experiment,RDN-B,TreeBoostler,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD
0,9_nell_sports_nell_finances,-0.303 +/- 0.006,-0.307 +/- 0.003,-0.366 +/- 0.001,-0.365 +/- 0.002,-0.365 +/- 0.001,-0.368 +/- 0.003
1,10_nell_finances_nell_sports,-0.085 +/- 0.001,NaN +/- NaN,-0.373 +/- 0.000,-0.344 +/- 0.004,-0.374 +/- 0.008,-0.370 +/- 0.008


# Results for AUC ROC

,Experiment,RDN-B,TreeBoostler,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD
0,9_nell_sports_nell_finances,0.762 +/- 0.041,0.746 +/- 0.019,0.558 +/- 0.000,0.562 +/- 0.003,0.540 +/- 0.002,0.541 +/- 0.003
1,10_nell_finances_nell_sports,0.994 +/- 0.001,NaN +/- NaN,0.484 +/- 0.000,0.759 +/- 0.000,0.487 +/- 0.004,0.489 +/- 0.006


# Results for AUC PR

,Experiment,RDN-B,TreeBoostler,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD
0,9_nell_sports_nell_finances,0.089 +/- 0.017,0.092 +/- 0.009,0.002 +/- 0.000,0.002 +/- 0.000,0.002 +/- 0.000,0.002 +/- 0.000
1,10_nell_finances_nell_sports,0.313 +/- 0.047,NaN +/- NaN,0.002 +/- 0.000,0.005 +/- 0.000,0.002 +/- 0.000,0.002 +/- 0.000


# Results for Learning time

,Experiment,RDN-B,TreeBoostler,Transfer Learning - SoftCosine,Transfer Learning - Euclidean,Transfer Learning - WMD,Transfer Learning - Relax-WMD
0,9_nell_sports_nell_finances,42.075 +/- 9.081,278.228 +/- 97.144,8.447 +/- 0.044,7.439 +/- 0.510,7.442 +/- 0.150,46.458 +/- 0.165
1,10_nell_finances_nell_sports,349.809 +/- 11.872,NaN +/- NaN,28.553 +/- 0.451,20.899 +/- 0.329,17.750 +/- 5.045,59.159 +/- 1.520
